# ch05 실습 예제 확장하기

## 웹 사이트를 어떻게 구성할 것인가?

1. 프로젝트 뼈대 만들기
2. 애플리케이션 설계하기
3. 애플리케이션 - Model 코딩하기
4. 애플리케이션 - URLconf 코딩하기
5. 애플리케이션 - Template 코딩하기
6. 애플리케이션 - View 코딩하기

In [2]:
%cd ch3

/Users/re4lfl0w/Documents/ipython/books/django_python_web_programming/ch3


In [3]:
%ll

total 288
-rw-r--r--   1 re4lfl0w  staff  143360  4 16 06:49 db.sqlite3
-rwxr-xr-x   1 re4lfl0w  staff     249  4 16 01:19 manage.py*
drwxr-xr-x  10 re4lfl0w  staff     340  4 16 06:12 mysite/
drwxr-xr-x  14 re4lfl0w  staff     476  4 16 05:55 polls/
drwxr-xr-x   3 re4lfl0w  staff     102  4 16 05:57 templates/


In [4]:
!python manage.py startapp books

In [5]:
!tree

.
├── books
│   ├── __init__.py
│   ├── admin.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── tests.py
│   └── views.py
├── db.sqlite3
├── manage.py
├── mysite
│   ├── __init__.py
│   ├── __init__.pyc
│   ├── settings.py
│   ├── settings.pyc
│   ├── urls.py
│   ├── urls.pyc
│   ├── wsgi.py
│   └── wsgi.pyc
├── polls
│   ├── __init__.py
│   ├── __init__.pyc
│   ├── admin.py
│   ├── admin.pyc
│   ├── models.py
│   ├── models.pyc
│   ├── templates
│   │   └── polls
│   │       ├── detail.html
│   │       ├── index.html
│   │       └── results.html
│   ├── tests.py
│   ├── urls.py
│   ├── urls.pyc
│   ├── views.py
│   └── views.pyc
└── templates
    └── admin
        └── base_site.html

8 directories, 31 files


### 5.1.2 애플리케이션 설계하기

### 표 5-1 books 애플리케이션 - Book 테이블 설계

컬럼명 | 타입 | 제약조건 | 설명
--- | --- | --- | ---
id | integer | NotNull, PK, AutoIncrement | Primary Key
title | varchar(100) | NotNull | 책 제목
authors | integer | NotNull, FK(Author.id), index | Many-To-Many
publisher | integer | NotNull, FK(Publisher.id), index | Foreign Key
publication_date | date | NotNull | 책 출판일

### 표 5-2 books 애플리케이션 Author 테이블 설계

컬럼명 | 타입 | 제약조건 | 설명
--- | --- | --- | ---
id | integer | NotNull, PK, AutoIncrement | Primary Key
salutation | varchar(100) | NotNull | 저자 인사말
name | varchar(50) | NotNull | 저자 성명
email | email | NotNull | 저자 이메일

### 표 5-3 books 애플리케이션 Publisher 테이블 설계

컬럼명 | 타입 | 제약조건 | 설명
--- | --- | --- | ---
id | integer | NotNull, PK, AutoIncrement | Primary Key
name | varchar(50) | NotNull | 출판사 이름
address | varchar(200) | NotNull | 출판사 주소
website | url | NotNull | 출판사 홈페이지

### 그림 5-3 books 애플리케이션 - 뷰 흐름 설계

URL | View | Template
--- | --- | ---
/books/ -> | BooksModelView.as_view() -> | index.html
/books/book/ -> | BookList.as_view() -> | book_list.html
/books/book/3/ -> | BookDetail.as_view() -> | book_detail.html

In [8]:
!python manage.py makemigrations

Migrations for 'books':
  0001_initial.py:
    - Create model Author
    - Create model Book
    - Create model Publisher
    - Add field publisher to book


In [9]:
!python manage.py migrate

Operations to perform:
  Synchronize unmigrated apps: polls
  Apply all migrations: admin, contenttypes, books, auth, sessions
Synchronizing apps without migrations:
  Creating tables...
  Installing custom SQL...
  Installing indexes...
Running migrations:
  Applying books.0001_initial... OK


In [10]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	books/migrations/0001_initial.py

nothing added to commit but untracked files present (use "git add" to track)


### 5.1.5 애플리케이션 - Template 코딩하기

### 표 5-4 books 애플리케이션 - URL/뷰/템플릿 매핑

URL 패턴 | 뷰 클래스명 | 템플릿 파일명 | 템플릿 설명
--- | --- | --- | ---
/books/ | BooksModelView | index.html | books 애플리케이션 첫 화면
/books/book/ | BookLIst | book_list.html | 책의 리스트를 보여줌
/books/author/ | AuthorList | author_list.html | 저자의 리스트를 보여줌
/books/publisher/ | PublisherList | publisher_list.html | 출판사의 리스트를 보여줌
/books/book/3/ | BookDetail | book_detail.html | 특정 책의 상세 정보를 보여줌
/books/author/3/ | AuthorDetail | author_detail.html | 특정 저자의 상세 정보를 보여줌
/books/publisher/3/ | PublisherDetail | publisher_detail.html | 특정 출판사의 상세 정보를 보여줌

- 부모 템플릿 역할을 하는 base.html 및 base_books.html 템플릿은 다음절에서 살펴보자

### 5.1.6 애플리케이션 - Template 상속 기능 추가

### 5.1.7 애플리케이션 - 클래스형 View 코딩하기

- 클래스형 뷰 또는 클래스형 generic view는 장고 사용시 매우 중요한 기능이고, 이번장의 핵심
- TemplateView
  - template_name 필수적으로 오버라이딩해서 지정
  - get_context_data(): 템플릿 시스템으로 넘겨줄 컨텍스트 변수가 있는 경우
  - get_context_data(): 메소드를 정의할 떄는 반드시 첫 줄에 super() 메소드를 호출해야 함
- ListView
  - 객체가 들어있는 리스트를 구성해서 이를 컨텍스트 변수로 템플릿 시스템에 넘겨주면 됨
  - 만일 이런 리스트를 테이블에 들어있는 모든 레코드를 가져와 구성하는 경우에는 테이블명, 즉 모델 클래스명만 지정해주면 됨
  - default: 켄텍스트 변수로 **object\_list** 사용, 템플릿 파일을 **모델명 소문자\_list.html**
  - Book 테이블로부터 모든 레코드를 가져와 object_list 라는 컨텍스트 변수 구성. 템플릿 파일은 디폴트로 books/book_list.html 파일이 됨
- DetailView
  - 특정 객체 하나를 컨텍스트 변수에 담아서 템플릿 시스템에 넘겨주면 됨
  - 만일 테이블에서 Primary Key로 조회해서 특정 객체를 가져오는 경우에는 테이블명, 즉 모델 클래스명만 지정해주면 됨. 조회시 사용할 **Primary Key** 값은 **URLconf 에서 추출하여 뷰로 넘어온 파라미터**를 사용

#### Generic View의 장점

- 데이터베이스 객체의 리스트를 보여주거나
- 특정 객체의 상세 내용을 보여주는 작업을 코딩할 때

#### 직접 코딩

1. 테이블에 접속
2. 쿼리 조건을 지정
3. 테이블로부터 가져온 결과를 컨텍스트 변수에 담기
4. 담긴 변수를 템플릿 시스템에 넘겨주기

## 5.2 프로젝트 첫 페이지 만들기

1. URLconf 코딩
2. Template 코딩
3. View 코딩

## 5.3 polls 애플리케이션 - 클래스형 뷰로 변경하기

### 5.3.1 URLconf 코딩하기

### 표 5-8 polls 애플리케이션 변경 - URL과 클래스형 뷰 매핑

URL 패턴 | 기존 뷰 이름<br/>(함수형 뷰) | 새로운 뷰 이름<br/>(클래스형 뷰) | 변경사항(템플릿 파일명은 동일함)
--- | --- | --- | ---
/polls/ | index() | IndexView | 뷰와 템플릿 모두 변경함(index.html)
/polls/5/ | detail() | DetailView | 뷰와 템플릿 모두 변경함(detail.html)
/polls/5/results/ | results() | ResultsView | 뷰와 템플릿 모두 변경함(results.html)
/polls/5/vote/ | vote() | vote() | 뷰와 템플릿 모두 변경사항 없음

### 5.3.2 Template 코딩하기

- base.html 을 상속받는 base_polls.html 템플릿을 만들고
- 기존 각 템플릿 파일에서 base_polls.html 템플릿을 상속받으면 됨.

### 5.3.3 View 코딩하기

- 함수형 뷰에서 클래스형 뷰로 변경되는 사항은 그보다 많은 편
- 그런데 클래스형 지네릭 뷰를 사용할 것이므로 코딩량은 오히려 줄어듦
- 클래스형 뷰를 코딩할 때 가장 먼저 고려해야 할 사항: 어떤 지네릭 뷰를 사용할 것이냐?

### 표 5-9 polls 애플리케이션 변경 - 지네릭 뷰 선택

URL 패턴 | 기존 뷰 이름<br/>(함수형 뷰) | 새로운 뷰 이름<br/>(클래스형 뷰) | 지네릭 뷰 선택
--- | --- | --- | ---
/polls/ | index() | IndexView | 질문 리스트를 보여주는 로직이므로, ListView를 사용함
/polls/5/ | detail() | DetailView | 질문 하나에 대한 세부 정보를 보여주는 로직이므로, DetailView를 사용함
/polls/5/results/ | results() | ResultsView | 투표 결과도 각 질문에 대한 세부 정보에 해당하므로, DetailView를 사용함
/polls/5/vote/ | vote() | vote() | 뷰와 템플릿 모두 변경사항 없음

```python
class IndexView(ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        '''Return the last five published questions.'''
        return Question.objects.order_by('-pub_date')[:5]
```

- 만일 이런 리스트를 테이블에 들어있는 모든 레코드를 가져와 구성하는 경우에는 테이블명, 즉 모델 클래스명만 지정해주면 됩니다. 그렇지 않은 경우에는 get_queryset() 메소드를 오버라이딩으로 정의하여 원하는 리스트를 구성해주면 됨. 켄텍스트 변수명과 템플릿 파일명은 디폭트 값을 사용할 수도 있고, 명시적으로 지정해줄 수도 있음
- 템플릿 파일명을 디폴트로 사용하지 않고, polls/index.html 로 지정
- 컨텍스트 변수명을 디포트로 사용하지 않고, latest_question_list 로 지정


### 5.3.4 로그 추가

- 로그를 남기기 위해서는 우선 settings.py 파일에 로깅 설정을 해줘야 함
- 로그 기록을 원하는 곳에 로깅 메소드를 호출하면 됨

#### 로거 개념

- 로깅을 선언해주고 version, disable_existing_loggers, formatters, handlers, loggers 등 만듦
- handlers: 설정할 때 제어하는 도구
- loggers: handler를 받아서 최종적으로 쓰이는 놈
- 그러니 **loggers에는 어떤 핸들러를 사용할지 반드시 적어놓아야 함**
- logs 디렉토리 꼭 만들어주기.. 자동으로 만들어주지 않는다!

In [12]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	books/migrations/0001_initial.py
	logs/

nothing added to commit but untracked files present (use "git add" to track)
